# Neptune + fastai

## Introduction

This guide will show you how to:

* Log basic metadata using `NeptuneCallback()`,
* Single and multi phase logging using `NeptuneCallback()`,
* Log model weights using `NeptuneCallback()`,
* Log images to a Neptune `run` using the `neptune` client.

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

* If you are running the notebook on your local machine, you need to have [Python](https://www.python.org/downloads/) and [pip](https://pypi.org/project/pip/) installed.
* If you want to see the example recorded to your own workspace instead:
    * Create a Neptune account → [Take me to registration](https://neptune.ai/register)
    * Create a Neptune project that you will use for tracking metadata → [Tell me more about projects](https://docs.neptune.ai/administration/projects)

## Install Neptune and dependencies

In [ ]:
! pip install fastai == 2.3.1 neptune-client[fastai]

## Basic Example

**Import libraries**

In [ ]:
import neptune.new as neptune

from fastai.vision.all import *
from neptune.new.integrations.fastai import NeptuneCallback

### Create a Neptune run

To log metadata to the Neptune project, you need the `project name` and the `api_token`.

To make this example easy to follow, we have created a public project **'common/fastai-integration'** and a shared user **'neptuner'** with the API token **'ANONYMOUS'**. As you will see in the code cell below.

**(Optional)** To log to your Neptune project:

* [Create a Neptune account](https://app.neptune.ai/register/)

* [Find your API token](https://docs.neptune.ai/getting-started/installation#authentication-neptune-api-token)
* [Find your project name](https://docs.neptune.ai/getting-started/installation#setting-the-project-name)

Pass your credentials to project and api_token arguments of neptune.init()

`run = neptune.init(api_token="YOUR_API_TOKEN", project="YOUR_WORKSPACE/YOUR_PROJECT")` # pass your credentials


In [ ]:
run = neptune.init(project="common/fastai-integration", api_token="ANONYMOUS")

**Dataset**

In [ ]:
path = untar_data(URLs.MNIST_TINY)

In [ ]:
dls = ImageDataLoaders.from_csv(path)

In [ ]:
dls.show_batch()

### Log metadata using `NeptuneCallback()`
Using NeptuneCallback the following is automatically logged to Neptune UI for you:
- Hyperparameters
- Loss
- Metrics
- Best model weights 
- Model architecture

In [ ]:
learn = cnn_learner(dls, resnet18, cbs=[NeptuneCallback(run, "experiment")])

In [ ]:
learn.fit_one_cycle(1)

## More options

### Single and Multi phase logging

#### Log on a single training phase

In [ ]:
learn = cnn_learner(dls, resnet18)

In [ ]:
learn.fit_one_cycle(3, cbs=[NeptuneCallback(run, "bs", save_model_freq=2)])

#### Log all training phases of the learner

In [ ]:
learn = cnn_learner(dls, resnet18, cbs=[NeptuneCallback(run, "experiment")])

In [ ]:
learn.fit_one_cycle(2)

In [ ]:
learn.fit_one_cycle(2)

### Model weights

1. By default NeptuneCallback() saves and logs the best model for you automatically. 
You can disable it by setting `save_best_model` arg to False.

2. Log Every N epochs

In [ ]:
n = 4
learn = cnn_learner(dls, resnet18, cbs=[NeptuneCallback(run, "experiment", save_model_freq=n)])

In [ ]:
learn.fit_one_cycle(8)

 3. Add SaveModelCallback 

  If you want to log your model weight files during single training phase then add  SavemodelCallback().

In [ ]:
learn.fit_one_cycle(1, cbs=[SaveModelCallback(), NeptuneCallback(run, "experiment")])

### Log images

In [ ]:
batch = dls.one_batch()
for i, (x, y) in enumerate(dls.decode_batch(batch)):
    run["images/one_batch"].log(File.as_image(x), name=f"{y}")

## Stop logging

<font color=red>**Warning:**</font><br>
Once you are done logging, you should stop tracking the run using the `stop()` method.
This is needed only while logging from a notebook environment. While logging through a script, Neptune automatically stops tracking once the script has completed execution.

In [ ]:
run.stop()